# Введение в MapReduce модель на Python


In [56]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [57]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [58]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [59]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [60]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [61]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [62]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [63]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [64]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [65]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [66]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [67]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [68]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [69]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [70]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 0.4939995854979272),
 (1, 0.4939995854979272),
 (2, 0.4939995854979272),
 (3, 0.4939995854979272),
 (4, 0.4939995854979272)]

## Inverted index

In [71]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [72]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [73]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [74]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('it', 18), ('what', 10)]),
 (1, [('a', 2), ('is', 18)])]

## TeraSort

In [75]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.01691614592825874),
   (None, 0.027439415714327953),
   (None, 0.13496214430030895),
   (None, 0.15400285152032245),
   (None, 0.23339130593934931),
   (None, 0.2652849566684271),
   (None, 0.30295119926095904),
   (None, 0.3095936909373269),
   (None, 0.3110522266406457),
   (None, 0.33927878733380235),
   (None, 0.36033538764785444),
   (None, 0.3758408330678269),
   (None, 0.3858900109461094),
   (None, 0.40369015080757664),
   (None, 0.4084982505776372),
   (None, 0.44284133943166915),
   (None, 0.4430009036162783),
   (None, 0.4523794967732343),
   (None, 0.464128596882619)]),
 (1,
  [(None, 0.5396899696385483),
   (None, 0.5918962419225595),
   (None, 0.6584882925777772),
   (None, 0.7158558547548473),
   (None, 0.7256404445089114),
   (None, 0.7323609712505912),
   (None, 0.7441343006134729),
   (None, 0.8209003185559157),
   (None, 0.8407932523537701),
   (None, 0.9293478056421852),
   (None, 0.983961085234929)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [76]:
# Создает пару ключ-значение (ключ равен 0 для всех пользователей)
def MAP_MAX(_, row: NamedTuple):
    yield (0, row)


# Находит пользователя с наибольшим числом социальных контактов
# среди всех пользователей с одинаковым ключом
def REDUCE_MAX(_, rows: Iterator[NamedTuple]):
    max_contacts = 0
    max_contacts_user = None
    for row in rows:
        if row.social_contacts > max_contacts:
            max_contacts = row.social_contacts
            max_contacts_user = row
    yield max_contacts_user

# Возвращаем список кортежей
def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
output

[User(id=3, age=33, social_contacts=800, gender='female')]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [77]:
# Создает пару ключ-значение (ключ равен 0 для всех пользователей)
def MAP_AVG(_, row: NamedTuple):
    yield (0, row)


# Нахождение среднего значения
def REDUCE_AVG(_, rows: Iterator[NamedTuple]):
    summ_counts = 0 # сумма social_contacts
    count_users = 0 # общее количество пользователей
    for row in rows:
        summ_counts += row.social_contacts
        count_users += 1
    if count_users > 0:
        yield summ_counts / count_users
    else:
        yield 0

# Возвращаем список кортежей
def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_AVG, REDUCE_AVG)
output = list(output)
output

[390.0]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [78]:
# Возвращает список кортежей (ключ, список значений)
def groupbykey(iterable):
    # сортируем по ключу
    iterable = sorted(iterable, key=lambda e: e[0])
    dict_t = {}
    # ключи в словаре хранятся уникальные
    for k2, v2 in iterable:
        dict_t[k2] = dict_t.get(k2, []) + [v2]
    return dict_t.items()

# находим максимальное значение в каждой группе с одинаковым ключом
output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
print(f"max: {output}")

# находим среднее значение в каждой группе с одинаковым ключом
output = MapReduce(RECORDREADER, MAP_AVG, REDUCE_AVG)
output = list(output)
print(f"average: {output}")


max: [User(id=3, age=33, social_contacts=800, gender='female')]
average: [390.0]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [79]:
# исходные данные пользователей

input_collection = [
    User(id=0, age=44, gender="male", social_contacts=20),
    User(id=0, age=35, gender="female", social_contacts=239),
    User(id=1, age=26, gender="male", social_contacts=638),
    User(id=2, age=17, gender="male", social_contacts=134),
    User(id=2, age=37, gender="female", social_contacts=234),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=3, age=34, gender="female", social_contacts=756),
]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

# Возвращает список кортежей
def groupbykey(iterable):
    dict_t = {}
    for k2, v2 in iterable:
        dict_t[k2] = dict_t.get(k2, []) + [v2]
    return dict_t.items()

# Группируем по ключу
def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for k2, v2 in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [
        (partition_id, sorted(partition.items(), key=lambda x: x[0]))
        for (partition_id, partition) in enumerate(partitions)
    ]

# Определяем номер раздела для объекта
def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers


def MapReduceDistributed(
    INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None
):
    map_partitions = map(
        lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)),
        INPUTFORMAT(),
    )
    if COMBINER != None:
        map_partitions = map(
            lambda map_partition: flatten(
                map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))
            ),
            map_partitions,
        )
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle
    reduce_outputs = map(
        lambda reduce_partition: (
            reduce_partition[0],
            flatten(
                map(
                    lambda reduce_input_group: REDUCE(*reduce_input_group),
                    reduce_partition[1],
                )
            ),
        ),
        reduce_partitions,
    )

    print(
        "{} key-value pairs were sent over a network.".format(
            sum(
                [
                    len(vs)
                    for (k, vs) in flatten(
                        [partition for (partition_id, partition) in reduce_partitions]
                    )
                ]
            )
        )
    )
    return reduce_outputs

maps = 3
reducers = 2

# Разделяем на порции, с которыми будут работать мапперы
def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split):
            yield (userindex, user)

    # разделеляем массив пользователей для вычислительных узлов
    split_size = int(np.ceil(len(input_collection) / maps))
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])


# Возвращаем кортеж с id пользователя и самим пользователем
def MAP_UNIQUE(_, user: str):
    yield (user.id, user)


# Возвращаем только первого пользователя из группы с одинаковым id
def REDUCE_UNIQUE(user_id: int, users: Iterator[int]):
    yield users[0]


partitioned_output = MapReduceDistributed(
    INPUTFORMAT, MAP_UNIQUE, REDUCE_UNIQUE, COMBINER=None
)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]
partitioned_output


7 key-value pairs were sent over a network.


[(0,
  [User(id=0, age=44, social_contacts=20, gender='male'),
   User(id=2, age=17, social_contacts=134, gender='male')]),
 (1,
  [User(id=1, age=26, social_contacts=638, gender='male'),
   User(id=3, age=31, social_contacts=521, gender='male')])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [80]:
# Проверяем значение параметра gender
def MAP_SELECT(_, row: NamedTuple):
  # предикатом будет проверка параметра gender
    if row.gender == "female":
        yield (row, row)

# Возвращаем ключ и все строки для этого ключа
def REDUCE_SELECT(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)

# Создаем список кортежей
def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_SELECT, REDUCE_SELECT)
output = list(output)
output


[(User(id=0, age=35, social_contacts=239, gender='female'),
  [User(id=0, age=35, social_contacts=239, gender='female')]),
 (User(id=2, age=37, social_contacts=234, gender='female'),
  [User(id=2, age=37, social_contacts=234, gender='female')]),
 (User(id=3, age=34, social_contacts=756, gender='female'),
  [User(id=3, age=34, social_contacts=756, gender='female')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [81]:
# исходные данные пользователей
input_collection_2 = [
    dict(id=0, age=44, gender="male", social_contacts=20),
    dict(id=0, age=35, gender="female", social_contacts=239),
    dict(id=1, age=26, gender="undefined", social_contacts=638),
    dict(id=2, age=17, gender="male", social_contacts=134),
    dict(id=2, age=37, gender="undefined", social_contacts=234),
    dict(id=3, age=31, gender="male", social_contacts=521),
    dict(id=3, age=34, gender="female", social_contacts=756),
]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    dict_t = {}
    for k2, v2 in iterable:
        dict_t[k2] = dict_t.get(k2, []) + [v2]
    return dict_t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )

# Возвращаем пару ключ-значение (ключ id)
def MAP_PROJECTION(_, row: dict):
    S = ["male", "female"]
    if row["gender"] in S:
        yield (row["id"], row)
    else:
        # Удаляем поле gender у полученого словаря
        modified_row = row.copy()
        del modified_row["gender"]
        yield (modified_row["id"], modified_row)


def REDUCE_PROJECTION(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u["id"], u) for u in input_collection_2]


output = MapReduce(RECORDREADER, MAP_PROJECTION, REDUCE_PROJECTION)
output = list(output)
output


[(0,
  [{'id': 0, 'age': 44, 'gender': 'male', 'social_contacts': 20},
   {'id': 0, 'age': 35, 'gender': 'female', 'social_contacts': 239}]),
 (1, [{'id': 1, 'age': 26, 'social_contacts': 638}]),
 (2,
  [{'id': 2, 'age': 17, 'gender': 'male', 'social_contacts': 134},
   {'id': 2, 'age': 37, 'social_contacts': 234}]),
 (3,
  [{'id': 3, 'age': 31, 'gender': 'male', 'social_contacts': 521},
   {'id': 3, 'age': 34, 'gender': 'female', 'social_contacts': 756}])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [82]:
# исходные данные пользователей
input_collection_a = [
    User(id=1, age=54, gender="male", social_contacts=20),
    User(id=2, age=25, gender="female", social_contacts=239),
    User(id=3, age=27, gender="male", social_contacts=638),
    User(id=4, age=16, gender="male", social_contacts=137),
    User(id=5, age=35, gender="female", social_contacts=247),
]

input_collection_b = [
    User(id=5, age=35, gender="female", social_contacts=247),
    User(id=6, age=31, gender="male", social_contacts=521),
    User(id=7, age=32, gender="female", social_contacts=749),
]

# Генерируем пару ключ-значение (ключ - id)
def MAP_UNION(_, row: NamedTuple):
    yield (row.id, row)


# Независимо от присутствия пользователя возвращаем (t, t)
def REDUCE_UNION(row: str, rows: Iterator[NamedTuple]):
    yield (rows[0], rows[0])


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION)
output = list(output)
output

[(User(id=1, age=54, social_contacts=20, gender='male'),
  User(id=1, age=54, social_contacts=20, gender='male')),
 (User(id=2, age=25, social_contacts=239, gender='female'),
  User(id=2, age=25, social_contacts=239, gender='female')),
 (User(id=3, age=27, social_contacts=638, gender='male'),
  User(id=3, age=27, social_contacts=638, gender='male')),
 (User(id=4, age=16, social_contacts=137, gender='male'),
  User(id=4, age=16, social_contacts=137, gender='male')),
 (User(id=5, age=35, social_contacts=247, gender='female'),
  User(id=5, age=35, social_contacts=247, gender='female')),
 (User(id=6, age=31, social_contacts=521, gender='male'),
  User(id=6, age=31, social_contacts=521, gender='male')),
 (User(id=7, age=32, social_contacts=749, gender='female'),
  User(id=7, age=32, social_contacts=749, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [83]:
# Группируем (по пользователям)
def MAP_INTERSECTION(_, row: NamedTuple):
    yield (row.id, row)


# Если пользователь присутствует в обеих выборках, то возвращаем его
def REDUCE_INTERSECTION(row_id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP_INTERSECTION, REDUCE_INTERSECTION)
output = list(output)
output

[[User(id=5, age=35, social_contacts=247, gender='female'),
  User(id=5, age=35, social_contacts=247, gender='female')]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [84]:
# Группируем (по пользователям)
def MAP_DIFFERENCE(collection_id, user):
    yield (user, collection_id)


# Возвращаем пользователей, которые входят в первую выборку и не входят во вторую
def REDUCE_DIFFERENCE(user, collections):
    if collections == [0]:
        yield (user)


# Генерируем список кортежей (на первом месте - номер выборки)
def RECORDREADER():
    return [(0, a) for a in input_collection_a] + [(1, b) for b in input_collection_b]


output = MapReduce(RECORDREADER, MAP_DIFFERENCE, REDUCE_DIFFERENCE)
output = list(output)
output

[User(id=1, age=54, social_contacts=20, gender='male'),
 User(id=2, age=25, social_contacts=239, gender='female'),
 User(id=3, age=27, social_contacts=638, gender='male'),
 User(id=4, age=16, social_contacts=137, gender='male')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [85]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    language_id: int

# коллекция первая
users_collection = [
    User(id=1, age=54, gender="male", social_contacts=20, language_id=1),
    User(id=2, age=25, gender="female", social_contacts=240, language_id=2),
    User(id=3, age=27, gender="male", social_contacts=642, language_id=2),
    User(id=4, age=16, gender="male", social_contacts=123, language_id=3),
    User(id=5, age=35, gender="female", social_contacts=247, language_id=4),
]


class Language(NamedTuple):
    id: int
    name: str

# коллекция вторая
language_collection = [
    Language(id=1, name="English"),
    Language(id=2, name="Spanish"),
    Language(id=3, name="German"),
    Language(id=4, name="French"),
    Language(id=5, name="Turkish"),
]

# Группировка пользователей по id языка
def MAP_JOIN(language_id, row):
    yield (language_id, row)


def REDUCE_JOIN(language_id, rows):
    users = []
    language  = None

    # определяем язык
    for row in rows:
        if type(row) is User:
            users += [row]
        else:
            language = row

    for row in rows:
        if type(row) is User:
            # возвращаем тройки (a,b,c), где a - пользователь, b - id языка, c - язык
            yield (row, row.language_id, language)

# Генерируем список кортежей (id языка, язык или пользователь)
def RECORDREADER():
    return [(user.language_id, user) for user in users_collection] + [
        (language.id, language) for language in language_collection
    ]


output = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
output = list(output)
joined_data = output
joined_data

[(User(id=1, age=54, social_contacts=20, gender='male', language_id=1),
  1,
  Language(id=1, name='English')),
 (User(id=2, age=25, social_contacts=240, gender='female', language_id=2),
  2,
  Language(id=2, name='Spanish')),
 (User(id=3, age=27, social_contacts=642, gender='male', language_id=2),
  2,
  Language(id=2, name='Spanish')),
 (User(id=4, age=16, social_contacts=123, gender='male', language_id=3),
  3,
  Language(id=3, name='German')),
 (User(id=5, age=35, social_contacts=247, gender='female', language_id=4),
  4,
  Language(id=4, name='French'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [86]:
# Группируем по id языка
def MAP_GROUP(language_id, user, language):
    yield (language_id, user)


# Находим количество пользователей в каждом из языков
def REDUCE_GROUP(language_id, rows):
    yield f"language with id={language_id} has {len(rows)} user(s)"


def RECORDREADER():
    return [(language_id, user, language) for user, language_id, language in joined_data]


output = MapReduce(RECORDREADER, MAP_GROUP, REDUCE_GROUP)
output = list(output)
output

['language with id=1 has 1 user(s)',
 'language with id=2 has 2 user(s)',
 'language with id=3 has 1 user(s)',
 'language with id=4 has 1 user(s)']

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [87]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [88]:
import numpy as np

# задаем значения компонентам
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)  # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J, K)



def RECORDREADER():
  # генерируем элементы второй матрицы с их индексами
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j, k), big_mat[j, k])


# Используем сгенерированные элементы 2-й матрицы и возвращаем
# их произведения между соответствующими элементами 1-й матрицы.
def MAP(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(I):
        k2 = (i, k)
        v2 = small_mat[i, j] * w
        yield (k2, v2)

# Суммируем полученные произведения
def REDUCE(key, values):
    (i, k) = key
    k3 = (i, k)
    v3 = 0
    for j in range(J):
        v3 += values[j]
    yield (k3, v3)

Проверьте своё решение

In [89]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [90]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [91]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)


# Генерируем пары элементов из матриц вместе с индексами, которые соответствуют им
def RECORDREADER():
    for i in range(I):
        for j in range(J):
            for k in range(K):
                yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))


# Перемножаем полученные элементы двух матриц
# И группируем произведения по индексу (в результирующей матрице)
def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)

# Суммируем полученные произведения для вычисления результирующей матрицы
def REDUCE(key, values):
    (i, k) = key
    k3 = (i, k)
    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)

# Проверим себя ниже
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)


def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat


np.allclose(reference_solution, asmatrix(solution))  # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [92]:
maps = 2
reducers = 2


def INPUTFORMAT():
    global maps

    # Генерируем пары элементов из обеих матриц
    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

    # По строкам первой матрицы происзводим разделение между вычислительными узлами
    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))


# Возвращаем произведения соответствующих элементов из обеих матриц
# И группируем их по индексу в результирующей матрице
def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)


# Суммируем полученные произведения
def REDUCE(key, values):
    (i, k) = key
    k3 = (i, k)
    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]

# Соединяем полученные произведения с разных вычислительных узлов
solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]
# Проверяем себя
print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))  # should return true

240 key-value pairs were sent over a network.
[[0.72349065 0.43745608 0.47359346 0.42930948 0.95575864 0.31769992
  0.28434783 1.02167551 0.72619718 1.06958981 0.86273571 0.5097025
  0.10226594 0.94871701 1.11911668 0.84670762 1.09138556 0.96577767
  0.35283332 1.07824859 0.66896702 1.12673701 0.77449634 0.33300847
  0.58343332 0.90849585 0.81482154 0.97108185 0.66252061 0.95104494
  0.26009097 1.3103634  0.53202327 0.94805064 1.32051586 0.6565518
  0.99724975 0.43730428 0.857062   0.72038694]
 [0.71163072 0.57311552 0.64739693 0.39686203 0.85276281 0.52305305
  0.31455424 0.72665734 0.4817421  0.86056542 0.8846199  0.56361425
  0.35689727 0.6891217  0.88936576 0.77131603 0.94150315 0.72541664
  0.48257646 1.05913198 0.56532165 1.13718965 0.66877568 0.41098748
  0.52344291 0.96591908 0.71560503 0.90302811 0.62739845 0.74444981
  0.39368795 1.26953265 0.75234896 0.9563454  1.11479957 0.67923082
  0.96829337 0.44133364 0.60344232 0.66639966]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Решение будет работать, так как каждая пара элементов, генерируемая RECORDREADER, идентифицируется по ее индексам. Также следует учитывать, что RECORDREADER-ы могут генерировать случайное пожмножество элементов в матрице (здесь мы для каждой матрицы реализуем свой RECORDREADER-ы).